# Tumor load calculation for the lymphoma cohort

You need to use the 2.2.39 version of moosez, which is installed by the following command:

```bash
pip install moosez==2.2.39
```
And the other steps are the same as described in the moose document.

The next thing is that the weights file cannot be downloaded. I can provide a local version of `Dataset789_Tumor_3/`, which could be unzipped at the `./models/nnunet_trained_models/` directory.

You also need to acquire the `imvis` package by Xiangxi Meng. I have not released it yet, and you may need to follow the instructions on the [GitHub repository](https://www.github.com/MengXiangxi/imvis). Remember to install this package into the same venv as the corresponding version of moosez.

In [ ]:
import imvis as iv
from imvis import utils
import os
import shutil
import glob
import pydicom
import dicom2nifti
import SimpleITK as sitk
import numpy as np
from pydicom import dcmread 


data_dir = "../data"



Set up the folders

In [ ]:
Patients = []
newdata_dir = '../data/External_Verification'
for i in os.listdir(os.path.join(data_dir,'0')):
    if os.path.isdir(os.path.join(data_dir,'0',i)):
        Patients.append(i)
        try:
            os.mkdir(os.path.join(newdata_dir,"input/0", i))
            os.mkdir(os.path.join(newdata_dir,"output/0", i))
        except:
            pass

In [ ]:
for i in Patients:
    imageName =  "PET"
    imagePath = os.path.join(data_dir,'dicomdata', i, imageName)
    dicom_path = os.path.join(imagePath,os.listdir(imagePath)[5])
    if i in os.listdir('../data/segdl/main'):
        nif_fname = os.path.join('../data/segdl/main',i,'PET.nii.gz')
    elif i in os.listdir('../data/segdl/vali'):
        nif_fname = os.path.join('../data/segdl/vali',i,'PET.nii.gz')
    # print(dicom_path)
    dcm = dcmread(dicom_path)
    radiopharm_datetime = dcm.RadiopharmaceuticalInformationSequence[0].RadiopharmaceuticalStartDateTime
    injection_dose = float(dcm.RadiopharmaceuticalInformationSequence[0].RadionuclideTotalDose)
    half_life = float(dcm.RadiopharmaceuticalInformationSequence[0].RadionuclideHalfLife)
    weight = dcm[0x0010,0x1030].value
    if dcm[0x0054,0x1102].value == 'START':
        try:
            acquisition_datetime = dcm[0x0008,0x002A].value
        except KeyError:
            acquisition_datetime = dcm[0x0008,0x0022].value +\
                dcm[0x0008,0x0032].value
        dose = injection_dose * 2**(-utils.datetimestr_diff(acquisition_datetime,radiopharm_datetime)/half_life)
    elif dcm[0x0054,0x1102].value == 'ADMIN':
        dose = injection_dose
    else:
        print('ERROR')
    SUVfactor = weight * 1000 /dose
    image = sitk.ReadImage(nif_fname)
    image = sitk.Cast(image,sitk.sitkFloat32)
    new_pet = image * SUVfactor
    sitk.WriteImage(new_pet,os.path.join(newdata_dir,"input", i, "PT.nii.gz"))

In [ ]:
from moosez import moose

for i in Patients:
    input_dir = os.path.join("../data/input", i)
    output_dir = os.path.join("../data/output", i)
    print(i)
    moose("clin_pt_fdg_tumor", input_dir, output_dir,"cuda")

Quantification of the results

In [ ]:
%matplotlib inline
with open("output.csv", "w") as f:
    f.write("Patient ID,Volume,Mean,Std,Tumor Load\n")

for i in Patients:
    pt_fname = glob.glob("PT*nii.gz", root_dir=os.path.join("input", i))[0]
    pt_nii = sitk.ReadImage(os.path.join("input", i, pt_fname))
    pt_img = sitk.GetArrayFromImage(pt_nii)
    mask_fname = glob.glob("*nii.gz", root_dir=os.path.join("output", i))[0]
    mask_nii = sitk.ReadImage(os.path.join("output", i, mask_fname))
    mask_img = sitk.GetArrayFromImage(mask_nii)
    spacing = pt_nii.GetSpacing()
    voxel_size = np.prod(spacing)
    voi = mask_img*pt_img
    voi = voi.flatten()
    voi = voi[mask_img.flatten()>0]
    if len(voi)>0:
        volume = np.sum(voi) * voxel_size
        mean = np.mean(voi)
        std = np.std(voi)
        tumorload = volume * mean
    else:
        volume = 0
        mean = 0
        std = 0
        tumorload = 0
    with open("output.csv", "a") as f:
        f.write("{},{},{},{},{}\n".format(i, volume, mean, std, tumorload))

In [ ]:
for i in os.listdir('../data/segdl/1'):
    imageName =  "PET"
    imagePath = os.path.join('../data/moosedata/dicomdata', i, imageName)
    dicom_path = os.path.join(imagePath,os.listdir(imagePath)[5])
    nif_fname = os.path.join('../data/segdl/1',i,'PET.nii.gz')
    # print(dicom_path)
    dcm = dcmread(dicom_path)
    radiopharm_datetime = dcm.RadiopharmaceuticalInformationSequence[0].RadiopharmaceuticalStartDateTime
    # radiopharm_datetime = dcm[0x0008,0x0022].value + radiopharm_datetime
    injection_dose = float(dcm.RadiopharmaceuticalInformationSequence[0].RadionuclideTotalDose)
    half_life = float(dcm.RadiopharmaceuticalInformationSequence[0].RadionuclideHalfLife)
    weight = dcm[0x0010,0x1030].value
    if dcm[0x0054,0x1102].value == 'START':
        try:
            acquisition_datetime = dcm[0x0008,0x002A].value
        except KeyError:
            acquisition_datetime = dcm[0x0008,0x0022].value +\
                dcm[0x0008,0x0032].value
        dose = injection_dose * 2**(-utils.datetimestr_diff(acquisition_datetime,radiopharm_datetime)/half_life)
    elif dcm[0x0054,0x1102].value == 'ADMIN':
        dose = injection_dose
    else:
        print('ERROR')
    SUVfactor = weight * 1000 /dose
    image = sitk.ReadImage(nif_fname)
    image = sitk.Cast(image,sitk.sitkFloat32)
    # data = sitk.GetArrayFromImage(image)
    new_pet = image * SUVfactor
    # out = sitk.GetImageFromArray(new_pet)
    sitk.WriteImage(new_pet,os.path.join("../data/segdl/1", i, "SUV.nii.gz"))

In [ ]:
import SimpleITK as sitk
import os
import joblib
for i in os.listdir('../data/0'):
    mask_moose = os.path.join('../data/output/0',i,'PT_FDG_Tumor_PT_0000.nii.gz')
    mask_lung = os.path.join('../data/0',i,'LUNG_mask.pkl')
    img_moose = sitk.ReadImage(mask_moose)
    array_moose = sitk.GetArrayFromImage(img_moose)
    array_lung = joblib.load(mask_lung)
    new_mask = array_moose
    new_mask[array_lung == 0] = 0
    sitk.WriteImage(new_pet,os.path.join("../data/0", i, "PET_M_P.nii.gz"))